In [40]:
import requests
import json
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np


Our list of targets

In [41]:
mytargets = ['DRD2', 'DRD4', 'PINK1', 'GRIN1']

Make the API call with our list of targets to find the associations. Set facets to true.

In [42]:
url = 'https://www.targetvalidation.org/api/latest/public/association/filter'
headers = {"Accept": "application/json"}
# There may be an easier way of building these parameters...
data = "{\"disease\":[\"EFO_0002508\"], \"size\":50}"

response = requests.post(url, headers=headers, data=data)
output = response.json()

Print out all the json returned just for reference

In [43]:
#print json.dumps(output, indent=2)

Run through all the targets and find the one's we're interested in

In [44]:
targets = []
symbols = []
associations = []

for target in output['data']:
    if target['target']['gene_info']['symbol'] in mytargets:
        symbols.append(target['target']['gene_info']['symbol'])
        
        scores = []
        scores.append(target['association_score']['overall'])
        scores.append(target['association_score']['datatypes']['genetic_association'])
        scores.append(target['association_score']['datatypes']['somatic_mutation'])
        scores.append(target['association_score']['datatypes']['known_drug'])
        scores.append(target['association_score']['datatypes']['affected_pathway'])
        scores.append(target['association_score']['datatypes']['rna_expression'])
        scores.append(target['association_score']['datatypes']['literature'])
        scores.append(target['association_score']['datatypes']['animal_model'])

        associations.append(scores)
        
        
#print symbols
#print associations

In [45]:
column_labels = symbols
row_labels = ['Association score', 'Genetic associations', 'Somatic mutations', 'Drugs', 'Affected pathways', 'RNA expression', 'Text mining', 'Animal models']
data = np.array(associations)
fig, ax = plt.subplots()
heatmap = ax.pcolor(data, cmap=plt.cm.Blues)
fig.subplots_adjust(top=0.7)

# put the major ticks at the middle of each cell
ax.set_xticks(np.arange(data.shape[1])+1, minor=False)
ax.set_yticks(np.arange(data.shape[0])+0.5, minor=False)

# want a more natural, table-like display
ax.invert_yaxis()
ax.xaxis.tick_top()

ax.set_xticklabels(row_labels, minor=False, rotation=60)
ax.set_yticklabels(column_labels, minor=False)
plt.show()

